In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 603.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.4/778.4 kB 5.2 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [1]:
from ultralytics import YOLO
import os

#### Trainning the model

In [ ]:
results = model.train(
    data= '/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=100,
    imgsz=640,
    patience=50,
    device=0,
    batch=64,
    optimizer= 'SGD',
    momentum=0.95
)

#### Evalution

In [5]:
model = YOLO("/kaggle/input/best-model/best_80_10_10.pt")  

metrics = model.val()  
metrics.box.map  
metrics.box.map50  
metrics.box.map75  
metrics.box.maps  

Ultralytics YOLOv8.2.22 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 15.3MB/s]
val: Scanning /kaggle/input/80-10-10-dataset/valid/labels... 1016 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1016/1016 [00:03<00:00, 293.06it/s]


val: WARNING ⚠️ Cache directory /kaggle/input/80-10-10-dataset/valid is not writeable, cache not saved.


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:15<00:00,  4.04it/s]


                   all       1016       1872      0.897      0.817      0.901      0.562
              Drowning       1016       1107      0.886      0.875      0.934      0.606
              Swimming       1016        707      0.856      0.851      0.886      0.508
          out of water       1016         58       0.95      0.724      0.885      0.571
Speed: 0.4ms preprocess, 3.7ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to runs/detect/val


array([    0.60647,     0.50794,     0.57074])

#### hyperparameter-tuning

- in our case SGD as optimizer is better that AdamW


In [9]:
model = YOLO("/kaggle/input/best-model/best_80_10_10.pt") 

- lr0 = [0.0005, **0.001**, 0.005]
- lrf = [0.01, **0.005**]
- batch = [16, **32**, 64]
- momentum = [0.85, **0.9**, 0.95]
- warmup_momentum = [**0.5**]
- warmup_bias_lr = [**0.05**, 0.1, 0.2]
- weight_decay = [0.0001, **0.0005**, 0.001]
- patience = [**30**, 50]
- imgsz = [**640**, 800, 1024]


#### Testing the model

In [2]:
from ultralytics.utils.plotting import Annotator
import cv2

In [3]:
model = YOLO("best_80_10_10.pt") 
video_path = 'sample_02.mp4'
cap = cv2.VideoCapture(video_path)

In [4]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    results = model(frame)
    
    for r in results:
        annotator = Annotator(frame)
        
        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])
    
    frame = annotator.result()
    cv2.imshow("Object Detection", frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

0: 640x384 (no detections), 99.3ms
Speed: 6.2ms preprocess, 99.3ms inference, 522.1ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 (no detections), 87.9ms
Speed: 1.7ms preprocess, 87.9ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 (no detections), 94.4ms
Speed: 1.8ms preprocess, 94.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 (no detections), 111.7ms
Speed: 3.6ms preprocess, 111.7ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 73.8ms
Speed: 1.4ms preprocess, 73.8ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 (no detections), 65.7ms
Speed: 1.6ms preprocess, 65.7ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 (no detections), 67.0ms
Speed: 1.3ms preprocess, 67.0ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 74.3ms
Speed: 1.4ms preprocess, 74.3ms inferen

#### activate the Tracker in Yolov8